In [ ]:
!mkdir -p /root/.config/kaggle
! mv kaggle.json /root/.config/kaggle/kaggle.json
!chmod 600 /root/.config/kaggle/kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia
!unzip udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.09GB/s]
Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [1]:
import pandas as pd
import numpy as np

## Exploración de datos y valores NAN

### cantidad de valores NAN por columna

In [ ]:
pd.read_csv('train.csv').isna().sum()

,0
ID,0
PERIODO,0
ESTU_PRGM_ACADEMICO,0
ESTU_PRGM_DEPARTAMENTO,0
ESTU_VALORMATRICULAUNIVERSIDAD,6287
ESTU_HORASSEMANATRABAJA,30857
FAMI_ESTRATOVIVIENDA,32137
FAMI_TIENEINTERNET,26629
FAMI_EDUCACIONPADRE,23178
FAMI_TIENELAVADORA,39773


### Valores únicos por columna

In [ ]:
df = pd.read_csv('train.csv')
df.nunique()

,0
ID,692500
PERIODO,9
ESTU_PRGM_ACADEMICO,948
ESTU_PRGM_DEPARTAMENTO,31
ESTU_VALORMATRICULAUNIVERSIDAD,8
ESTU_HORASSEMANATRABAJA,5
FAMI_ESTRATOVIVIENDA,7
FAMI_TIENEINTERNET,2
FAMI_EDUCACIONPADRE,12
FAMI_TIENELAVADORA,2


### Descripcción de los datos

In [ ]:
df._get_numeric_data().describe().T

,count,mean,std,min,25%,50%,75%,max
ID,692500.0,494606.130576,285585.209455,1.0,247324.750,494564.500,741782.500,989286.000
PERIODO,692500.0,20198.366679,10.535037,20183.0,20195.000,20195.000,20203.000,20213.000
coef_1,692500.0,0.268629,0.122130,0.0,0.203,0.240,0.314,0.657
coef_2,692500.0,0.259996,0.093480,0.0,0.212,0.271,0.309,0.487
coef_3,692500.0,0.262087,0.058862,0.0,0.254,0.276,0.293,0.320
coef_4,692500.0,0.262903,0.067944,0.0,0.255,0.285,0.303,0.332


## Lectura de datasets y asignación a variables

In [2]:
dtr = pd.read_csv("train.csv")
dts = pd.read_csv("test.csv")
lentr = len(dtr)
dtr.shape, dts.shape

((692500, 21), (296786, 20))

## Limpieza y reemplazo de valores NAN para la columna ESTU_HORASSEMANATRABAJA

In [3]:
def organizar_horas_trabajadas_estudiante(data, columna):

    # Valores posibles
    mapeo = {
        "0": 0,
        "menos de 10 horas": 1,
        "entre 11 y 20 horas": 2,
        "entre 21 y 30 horas": 3,
        "más de 30 horas": 4
    }

    # Rellenar NaNs con valores  válidos
    modas = data.groupby('FAMI_ESTRATOVIVIENDA')[columna].transform(lambda x: x.mode().iloc[0])
    data[columna] = data[columna].fillna(modas)

    # Pasar todo a minúsculas y quitar espacios extra
    data[columna] = data[columna].astype(str).str.strip().str.lower()

    # Aplicar mapeo numérico
    data[columna] = data[columna].map(mapeo)

    return data


## Limpieza y reemplzado de valores NAN para la columna ESTU_HORASSEMANATRABAJA utilizando dummies

In [4]:
def organizar_horas_trabajadas_estudiante_columna(data, columna):

    # Valores posibles ordenados
    mapeo = {
        "0": 0,
        "menos de 10 horas": 1,
        "entre 11 y 20 horas": 2,
        "entre 21 y 30 horas": 3,
        "más de 30 horas": 4
    }

    # Rellenar NaNs con valores cíclicos válidos
    modas = data.groupby('FAMI_ESTRATOVIVIENDA')[columna].transform(lambda x: x.mode().iloc[0])
    data[columna] = data[columna].fillna(modas)

    dummies = pd.get_dummies(data[columna], prefix="Trabaja").astype(int)
    data = data.drop(columns=[columna]).join(dummies)

    return data

## Limpieza y reemplazo de valores NAN para la columna ESTU_PRGM_ACADEMICO

In [14]:
def organizar_programa_estudiante(data, columna):
    # Diccionario con categorías y palabras clave asociadas
    categorias = {
        "arquitectura": ["arquite"],
        "matematica": ["matema", "estad"],
        "ingenieria": ["ingenier"],
        "salud": ["psicolog", "medicina", "enfermer", "fisioterap", "odontolog", "nutric", "bacteriolog", "terapia", "farmac", "quirurg", "salud"],
        "biologia": ["biolog", "microbiol", "bioing", "bioqui"],
        "docencia": ["licenc", "pedagog", "educac", "docen"],
        "comunicacion": ["comunic", "periodismo", "cine", "audiovisual", "publicidad", "medios"],
        "social": ["sociolog", "trabajo social", "antropolog", "crimin", "histori", "historia", "filosof", "teolog"],
        "derecho": ["juris", "derecho"],
        "politica": ["gobier", "politic", "relaciones internacionales"],
        "administracion": ["admin", "gestion", "logist", "negocios", "gerenc", "direccion"],
        "economia": ["econom", "finanzas", "comercio", "mercadeo", "marketing"],
        "arte": ["arte", "musica", "teatro", "dise", "diseñ", "danza", "fotogra", "moda", "literat", "escénic", "escenari"],
        "deporte": ["deport", "recreac", "actividad fisica", "cultura fisica", "entrenamiento"],
        "tecnologia": ["informatic", "sistemas", "multimedia", "software", "digital", "datos"],
        "industria": ["industrial", "productiv", "produccion", "mecanic", "electron", "electromec", "automatiz", "topograf", "geolog", "minas", "quimic"],
        "agro": ["agronom", "agroind", "zootec", "pecuar", "acui", "agropec", "agricol", "forest"],
        "idiomas": ["ingles", "idioma", "lengua", "lenguas", "frances", "traducc", "biling"],
        "turismo": ["turismo", "gastronom", "hotel", "cocina"],
        "otros": []
    }

    # Normaliza los textos
    data[columna] = data[columna].astype(str).str.lower().str.strip()

    # Función interna para aplicar a cada valor
    def clasificar(programa):
        for categoria, palabras_clave in categorias.items():
            if any(palabra in programa for palabra in palabras_clave):
                return categoria
        return "otras"

    # Aplicar categorización
    data[columna] = data[columna].apply(clasificar)

    # dummies = pd.get_dummies(data[columna], prefix="Programa").astype(int)
    # data = data.drop(columns=[columna]).join(dummies)

    return data  # ✅ ESTA LÍNEA ES OBLIGATORIA



##  Limpieza y reemplazo de valores NAN para la columna FAMI_ESTRATOVIVIENDA

In [6]:
def organizar_estratos(data, columna):
  data[columna] = (
      data[columna]
      .str.lower()
      .str.replace("sin estrato", "0", regex=False)
      .str.replace("estrato ", "", regex=False))

  moda = data[columna].mode().values[0]

  data[columna] = data[columna].fillna(moda)


  data[columna] = (
    data[columna]
    .astype(int))

  return data

## Limpieza y reemplzado de valores NAN para las columnas que tienen solo dos posibles valores

In [7]:
def organizar_columna_binaria(data, columna, valores):
    modas = data.groupby('FAMI_ESTRATOVIVIENDA')[columna].transform(lambda x: x.mode().iloc[0])
    data[columna] = data[columna].fillna(modas)

    data.loc[data[columna] == valores[0], columna] = 1
    data.loc[data[columna] == valores[1], columna] = 0

    return data


## Limpieza y reemplazo de valores NAN para la columna ESTU_VALORMATRICULAUNIVERSIDAD

In [38]:
def organizar_valor_matricula_en_una_columna(data, columna):
    mapeo = {
       'Más de 7 millones':7,
       'Entre 5.5 millones y menos de 7 millones':6,
       'Entre 4 millones y menos de 5.5 millones':5,
       'Entre 2.5 millones y menos de 4 millones':4,
       'Entre 1 millón y menos de 2.5 millones':3,
       'Entre 500 mil y menos de 1 millón':2,
       'Menos de 500 mil': 1,
       'No pagó matrícula':0
      }

    # Rellenar NaNs con valores cíclicos válidos
    modas = data.groupby('FAMI_ESTRATOVIVIENDA')[columna].transform(lambda x: x.mode().iloc[0])
    data[columna] = data[columna].fillna(modas)

    # Aplicar mapeo numérico
    data[columna] = data[columna].map(mapeo)

    return data


## Limpieza y reemplazo de valores NAN para la columna FAMI_EDUCACIONPADRE

In [37]:
def organizar_educacion_padres(data, columna):
  # Lista de estratos (como strings, si así están en el DataFrame)

  mapeo = {
    'No sabe':0,
    'Ninguno':1,
    'No Aplica':2,
    'Primaria incompleta':3,
    'Primaria completa':4,
    'Secundaria (Bachillerato) incompleta':5,
    'Secundaria (Bachillerato) completa':6,
    'Técnica o tecnológica incompleta':7,
    'Técnica o tecnológica completa':8,
    'Educación profesional incompleta':9,
    'Educación profesional completa':10,
    'Postgrado':11,
  }

  # Asignar uno por uno, reiniciando al llegar al final de la lista de estratos
  modas = data.groupby('FAMI_ESTRATOVIVIENDA')[columna].transform(lambda x: x.mode().iloc[0])
  data[columna] = data[columna].fillna(modas)

  # if columna == "FAMI_EDUCACIONMADRE":
  #   dummies = pd.get_dummies(data[columna], prefix="Edu-madre").astype(int)
  #   data = data.drop(columns=[columna]).join(dummies)

  #   return data

  # dummies = pd.get_dummies(data[columna], prefix="Edu-padre").astype(int)
  # data = data.drop(columns=[columna]).join(dummies)
  data[columna] = data[columna].map(mapeo)

  return data


## Limpieza y reemplazo de valores NAN para la columna PERIODO

In [10]:
def organizar_periodos(data, columna):
    mapeo = {
        20183:0,
        20184:0, 
        20194:1,
        20195:1,
        20196:1,
        20202:2,
        20203:2,
        20212:3,
        20213:3
    }

    data[columna] = data[columna].map(mapeo)

    return data

## Limpieza y reemplazo de valores NAN para la columna FAMI_EDUCACIONMADRE

## Análisis de la correclación entre las columnas de coeficientes

In [ ]:
coef_cols = ['coef_1', 'coef_2', 'coef_3', 'coef_4']
dtr_coef = dtr[coef_cols]

correlation_matrix = dtr_coef.corr()

print("Matriz de correlación entre las columnas 'coef':")
print(correlation_matrix)

Matriz de correlación entre las columnas 'coef':
          coef_1    coef_2    coef_3    coef_4
coef_1  1.000000 -0.420428  0.401190 -0.179439
coef_2 -0.420428  1.000000  0.356097  0.821443
coef_3  0.401190  0.356097  1.000000  0.726692
coef_4 -0.179439  0.821443  0.726692  1.000000


## Función donde se eliminan columnas innecesarias y se aplican las funciones de limpieza

In [46]:
def columna_comodidades(data, columnas=["FAMI_TIENELAVADORA", "FAMI_TIENEAUTOMOVIL", "FAMI_TIENEINTERNET", "FAMI_TIENECOMPUTADOR","ESTU_PAGOMATRICULAPROPIO"]):
    data['Comodidades'] = data[columnas].sum(axis=1)
    return data

In [50]:
def clean_data(data):
  columnas_inecesarias = [
      "ID",
      # "FAMI_TIENELAVADORA",
      "coef_4",
      # "coef_1",
      #"PERIODO",
      # "FAMI_TIENEAUTOMOVIL",
      "FAMI_TIENEINTERNET.1",
      "ESTU_PRIVADO_LIBERTAD",
      #"ESTU_HORASSEMANATRABAJA",
      # "FAMI_EDUCACIONPADRE",
      # "FAMI_EDUCACIONMADRE",
      #"ESTU_PRGM_ACADEMICO",
      #"ESTU_PRGM_DEPARTAMENTO",
      #"FAMI_TIENEINTERNET",
      #"FAMI_TIENECOMPUTADOR",
      #"ESTU_PAGOMATRICULAPROPIO",
      #"ESTU_VALORMATRICULAUNIVERSIDAD"
  ]
  data=data.drop(columns=columnas_inecesarias)

  data=organizar_estratos(data, "FAMI_ESTRATOVIVIENDA")
  data=organizar_periodos(data, "PERIODO")
  data=organizar_horas_trabajadas_estudiante(data, "ESTU_HORASSEMANATRABAJA")
  # data=organizar_programa_estudiante(data, "ESTU_PRGM_ACADEMICO")
  data=organizar_columna_binaria(data, "FAMI_TIENEINTERNET", ["Si", "No"])
  data=organizar_columna_binaria(data, "FAMI_TIENECOMPUTADOR", ["Si", "No"])
  data=organizar_columna_binaria(data, "FAMI_TIENELAVADORA", ["Si", "No"])
  data=organizar_columna_binaria(data, "FAMI_TIENEAUTOMOVIL", ["Si", "No"])
  data=organizar_columna_binaria(data, "ESTU_PAGOMATRICULAPROPIO", ["Si", "No"])

  data=organizar_valor_matricula_en_una_columna(data, "ESTU_VALORMATRICULAUNIVERSIDAD")
  data=organizar_educacion_padres(data, "FAMI_EDUCACIONPADRE")
  data=organizar_educacion_padres(data, "FAMI_EDUCACIONMADRE")
  data=columna_comodidades(data)
  data=data.drop(columns=["FAMI_TIENELAVADORA", "FAMI_TIENEAUTOMOVIL", "FAMI_TIENEINTERNET", "FAMI_TIENECOMPUTADOR","ESTU_PAGOMATRICULAPROPIO"])
  return data

In [51]:
source_cols = [i for i in dtr.columns if i!="RENDIMIENTO_GLOBAL"]
all_data = pd.concat((dtr[source_cols], dts[source_cols]))
all_data.index = range(len(all_data))
all_data = clean_data(all_data)

Xtr, ytr = all_data.iloc[:lentr].values, dtr["RENDIMIENTO_GLOBAL"].values
Xts      = all_data.iloc[lentr:].values
print (Xtr.shape, ytr.shape)
print (Xts.shape)

(692500, 12) (692500,)
(296786, 12)


In [52]:
all_data.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,Comodidades
0,3,ENFERMERIA,BOGOTÁ,6,1,3,7,11,0.322,0.208,0.310,4
1,3,DERECHO,ATLANTICO,4,0,3,8,7,0.311,0.215,0.292,2
2,2,MERCADEO Y PUBLICIDAD,BOGOTÁ,4,4,3,6,6,0.297,0.214,0.305,2
3,1,ADMINISTRACION DE EMPRESAS,SANTANDER,5,0,4,0,6,0.485,0.172,0.252,3
4,3,PSICOLOGIA,ANTIOQUIA,4,3,3,4,4,0.316,0.232,0.285,4


## Catboost Classifier

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.7 MB/s eta 0:00:00


In [72]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(Xtr,ytr, test_size=0.2)

model = CatBoostClassifier(
            iterations=1000,
            learning_rate=0.06,
            depth=6,
            l2_leaf_reg=10,
            verbose=100,
            cat_features=[0, 1, 2, 3, 4, 5, 6, 7, 11]
        )

model.fit(Xtrain, ytrain, eval_set=(Xtest, ytest), use_best_model=True)

model.score(Xtrain, ytrain), model.score(Xtest, ytest)

0:	learn: 1.3713925	test: 1.3712588	best: 1.3712588 (0)	total: 3.21s	remaining: 53m 29s
100:	learn: 1.1991716	test: 1.1953419	best: 1.1953419 (100)	total: 4m 58s	remaining: 44m 19s
200:	learn: 1.1908200	test: 1.1877186	best: 1.1877186 (200)	total: 10m 10s	remaining: 40m 27s
300:	learn: 1.1867136	test: 1.1847287	best: 1.1847287 (300)	total: 15m 24s	remaining: 35m 47s
400:	learn: 1.1845959	test: 1.1835407	best: 1.1835407 (400)	total: 20m 42s	remaining: 30m 56s
500:	learn: 1.1831299	test: 1.1827631	best: 1.1827631 (500)	total: 25m 58s	remaining: 25m 52s
600:	learn: 1.1818905	test: 1.1822220	best: 1.1822220 (600)	total: 31m 14s	remaining: 20m 44s
700:	learn: 1.1807587	test: 1.1818683	best: 1.1818683 (700)	total: 36m 31s	remaining: 15m 34s
800:	learn: 1.1796823	test: 1.1815915	best: 1.1815886 (798)	total: 41m 48s	remaining: 10m 23s
900:	learn: 1.1786935	test: 1.1813567	best: 1.1813567 (900)	total: 47m 5s	remaining: 5m 10s
999:	learn: 1.1777125	test: 1.1811117	best: 1.1811108 (998)	total: 52

(np.float64(0.45625270758122743), np.float64(0.4457545126353791))

In [42]:
pd.DataFrame({'ID': dts['ID'].values, 'RENDIMIENTO_GLOBAL': model.predict(Xts).reshape(296786,)}).to_csv('predict.csv', index=False)